<a href="https://colab.research.google.com/github/amalphonse/60DaysUdacity/blob/master/trusted_aggregator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Federated Learning with Trusted Aggregator.

In [1]:
!pip install syft

     |████████████████████████████████| 296kB 2.8MB/s 
     |████████████████████████████████| 389kB 51.3MB/s 
     |████████████████████████████████| 81kB 31.9MB/s 
     |████████████████████████████████| 204kB 53.6MB/s 
     |████████████████████████████████| 256kB 41.9MB/s 
     |████████████████████████████████| 460kB 50.0MB/s 
     |████████████████████████████████| 2.1MB 32.0MB/s 
     |████████████████████████████████| 51kB 22.5MB/s 
     |████████████████████████████████| 266kB 47.7MB/s 
     |████████████████████████████████| 122kB 55.2MB/s 
  Created wheel for zstd: filename=zstd-1.4.1.0-cp36-cp36m-linux_x86_64.whl size=1067078 sha256=456d20104f1f43b9956a970456cadcdf156f04c3be859eafb13ec1e9dd6c8723
  Stored in directory: /root/.cache/pip/wheels/66/3f/ee/ac08c81af7c1b24a80c746df669ea3cb37542d27877d66ccf4
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=43a20b5fb5935eb4bdbd01bee1d6f7aeaa8500df8b1d7c2a379468f7bdf44a60
  Stored in di

In [2]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0828 03:52:39.098412 139868692100992 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0828 03:52:39.112868 139868692100992 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
#create a couple of workers
bob = sy.VirtualWorker(hook, id ="bob")
alice = sy.VirtualWorker(hook,id="alice")
secure_worker = sy.VirtualWorker(hook,id="secure_worker")

In [5]:
bob.add_workers([alice,secure_worker])
alice.add_workers([bob,secure_worker])
secure_worker.add_workers([alice,bob])

W0828 03:56:15.646148 139868692100992 base.py:654] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0828 03:56:15.648034 139868692100992 base.py:654] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0828 03:56:15.648945 139868692100992 base.py:654] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0828 03:56:15.651941 139868692100992 base.py:654] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0828 03:56:15.653744 139868692100992 base.py:654] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0828 03:56:15.654735 139868692100992 base.py:654] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
#A toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]],requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]],requires_grad=True)

In [0]:
#get pointers to training data on each worker by sending some trainign data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

In [0]:
alice_data = data[0:2].send(alice)
alice_target = target[0:2].send(alice)

In [0]:
model = nn.Linear(2,1)

In [0]:
bobs_model = model.copy().send(bob)

In [0]:
bobs_model = model.copy().send(bob)
alice_model = model.copy().send(alice)

In [0]:
bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
alice_opt = optim.SGD(params=alice_model.parameters(), lr=0.1)

In [0]:
for i in range(10):
  bobs_opt.zero_grad()
  bobs_pred = bobs_model(bobs_data)
  bobs_loss = ((bobs_pred - bobs_target) **2).sum()
  bobs_loss.backward()
  bobs_opt.step()
  bobs_loss = bobs_loss.get().data
  bobs_loss
  alice_opt.zero_grad()
  alice_pred = alice_model(alice_data)
  alice_loss = ((alice_pred - alice_target) **2).sum()
  alice_loss.backward()
  alice_opt.step()
  alice_loss = alice_loss.get().data
  alice_loss

In [0]:
alice_model.move(secure_worker)

In [0]:
bobs_model.move(secure_worker)

In [22]:
#both bobs and alices model are on secure worker
secure_worker._objects

{29776746127: Parameter containing:
 tensor([[-0.5182, -0.0522]], requires_grad=True),
 53854560440: Parameter containing:
 tensor([0.0321], requires_grad=True),
 85727238970: Parameter containing:
 tensor([[-0.5182, -0.0522]], requires_grad=True),
 92417855109: Parameter containing:
 tensor([0.0321], requires_grad=True)}

In [0]:
#now we need to average them
model.weight.data.set_(((alice_model.weight.data + bobs_model.weight.data)/2).get())
model.bias.data.set_(((alice_model.weight.data + bobs_model.weight.data)/2).get())